Component: **Prompts**

**Prompt Template**: Prompt templates are pre-defined recipes for generating prompts for language models.\
\
There are 2 types of Prompt Templates:


1.   Basic Prompt Template
2.   Chat Prompt Template



**Basic Prompt Template**

In [ ]:
# !pip install openai
# !pip install langchain

In [ ]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

In [ ]:
openai_api_key = ''

In [ ]:
llm = OpenAI(openai_api_key=openai_api_key)

In [ ]:
template = """Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{input_text}```
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["input_text","style"],
    template=template,
)

In [ ]:
input_text = """
स्वागत है! कृपया आपकी सहायता कैसे कर सकते हैं, बताएं।
हमें आपकी मदद करके खुशी होगी, कृपया आपकी समस्या का विवरण दें।
"""

style = """English \
in a calm and respectful tone
"""

final_prompt = prompt.format(input_text=input_text, style=style)


In [ ]:
print(final_prompt)

Translate the text that is delimited by triple backticks
into a style that is English in a calm and respectful tone
.
text: ```
स्वागत है! कृपया आपकी सहायता कैसे कर सकते हैं, बताएं।
हमें आपकी मदद करके खुशी होगी, कृपया आपकी समस्या का विवरण दें।
```



In [ ]:
llm(final_prompt) #output

'\nWelcome! Please let us know how we can help you. We would be glad to assist you, please provide us with a description of your issue.'

**Chat Prompt Template**

This prompt templates are for chat models. The prompt to Chat Models is a list of chat messages.

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is Bob."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=openai_api_key) #chat model

messages = template.format_messages(
    user_input="What is your name?"
)

llm(messages)

AIMessage(content='My name is Bob. How can I assist you today?', additional_kwargs={}, example=False)

In [ ]:
output = llm(template.format_messages(user_input="Can you suggest me 5 best movies of Brad Pitt?"))

In [ ]:
output.content.split('\n')

['Certainly! Here are five highly acclaimed movies featuring Brad Pitt:',
 '',
 '1. Fight Club (1999) - A gripping psychological thriller about a disillusioned office worker who forms an underground fight club with a charismatic soap salesman.',
 '',
 "2. Inglourious Basterds (2009) - Quentin Tarantino's alternate history war film, where Pitt plays the leader of a group of Jewish-American soldiers on a mission to assassinate Nazi leaders during World War II.",
 '',
 '3. Se7en (1995) - A dark and intense crime thriller in which Pitt portrays a young detective assigned to track down a serial killer who bases his murders on the seven deadly sins.',
 '',
 '4. The Curious Case of Benjamin Button (2008) - A unique romantic drama that follows the life of a man who ages backward, with Pitt playing the titular character.',
 '',
 "5. Once Upon a Time in Hollywood (2019) - Directed by Quentin Tarantino, this film is set in 1969 Los Angeles and features Pitt as a stunt double to Leonardo DiCaprio'

**Output Parsers**: Output parsers are classes that help structure language model responses. \
\
The 2 main types of Output Parsers:


1.   Structured Output Parsers
2.   Pydantic Output Parsers
3.   You can explore more on :https://python.langchain.com/docs/modules/model_io/output_parsers/

**Structured Output Parser**

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [ ]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
llm = OpenAI(temperature=0,openai_api_key = openai_api_key)

In [ ]:
input = prompt.format_prompt(question="what's the capital of france?")
output = llm(input.to_string())

In [ ]:
output_parser.parse(output)

{'answer': 'Paris',
 'source': 'https://www.worldatlas.com/articles/what-is-the-capital-of-france.html'}

**Pydantic (JSON) parser**

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [ ]:
# Here's another example, but with a compound typed field.
class Cricketer(BaseModel):
    name: str = Field(description="name of a cricketer")
    team_names: List[str] = Field(description="list of names of IPL teams he was part of")

In [ ]:
Cricketer_query = "Generate the IPL teams record of Dinesh Kartik"

parser = PydanticOutputParser(pydantic_object=Cricketer)

In [ ]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
input = prompt.format_prompt(query=Cricketer_query)
output = llm(input.to_string())

In [ ]:
parser.parse(output)

Cricketer(name='Dinesh Kartik', film_names=['Delhi Daredevils', 'Kolkata Knight Riders', 'Kings XI Punjab', 'Mumbai Indians'])